In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

loader =  torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size= 64, shuffle= True)
images, labels = next(iter(loader))

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Bond_pavel\anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
images.shape

torch.Size([64, 1, 28, 28])

In [17]:
bob

<VirtualWorker id:bob #objects:11>

In [4]:
bob_images = images[:int(len(labels)/2)].send(bob)
alice_images = images[int(len(labels)/2):].send(alice)
bob_labels = labels[:int(len(labels)/2)].send(bob)
alice_labels = labels[int(len(labels)/2):].send(alice)

In [27]:
crypto_provider._objects

{2691977083: Parameter containing:
 tensor([[ 0.0107, -0.0076,  0.0025,  ...,  0.0285,  0.0042,  0.0350],
         [-0.0227,  0.0209,  0.0284,  ..., -0.0312, -0.0252, -0.0004],
         [-0.0090, -0.0122,  0.0210,  ...,  0.0283,  0.0166,  0.0258],
         ...,
         [-0.0158, -0.0184, -0.0268,  ...,  0.0207, -0.0151,  0.0182],
         [ 0.0034,  0.0090,  0.0227,  ...,  0.0328,  0.0104,  0.0134],
         [ 0.0347, -0.0265,  0.0101,  ..., -0.0135,  0.0167,  0.0224]],
        requires_grad=True),
 70807063449: Parameter containing:
 tensor([-2.5496e-02,  9.4158e-03, -1.7456e-02, -2.0934e-02,  2.7473e-02,
          3.3200e-02,  2.9049e-02, -2.1816e-02,  2.7428e-02,  3.4861e-02,
          2.9423e-02, -3.1399e-02, -1.1593e-02,  3.3127e-02, -2.7856e-02,
         -5.9057e-03, -2.7883e-02,  6.9829e-04,  3.1226e-02,  4.6041e-03,
         -3.2660e-02,  2.6626e-02,  2.6679e-02, -2.5777e-02, -7.8612e-03,
         -3.0777e-02, -3.4753e-02, -2.6955e-02,  1.3617e-02, -1.7610e-02,
         -1.404

In [20]:
bob_model.images

AttributeError: 'Net' object has no attribute 'images'

In [19]:
bob._objects

{84377323490: tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4

In [8]:
labels

tensor([0, 5, 3, 3, 3, 9, 2, 3, 5, 5, 2, 9, 6, 7, 4, 8, 2, 9, 0, 0, 8, 1, 5, 3,
        8, 0, 0, 8, 3, 3, 9, 1, 1, 2, 6, 8, 8, 7, 0, 4, 5, 7, 8, 6, 6, 7, 6, 6,
        3, 6, 6, 4, 8, 8, 0, 1, 6, 8, 3, 7, 3, 8, 3, 2])

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [10]:
#for i in range(10):
model = Net()
bob_model = model.copy().send(bob)
alice_model = model.copy().send(alice)
bob_opt = optim.SGD(params=bob_model.parameters(),lr=0.1)
alice_opt = optim.SGD(params=alice_model.parameters(),lr=0.1)

In [16]:
for i in range(10):


    bob_opt.zero_grad()
    bob_pred = bob_model(bob_images)
    bob_loss = F.nll_loss(bob_pred, bob_labels)
    bob_loss.backward()
    bob_opt.step()
    bob_loss = bob_loss.get()

    alice_opt.zero_grad()
    alice_pred = alice_model(alice_images)
    alice_loss = F.nll_loss(alice_pred, alice_labels)
    alice_loss.backward()
    alice_opt.step()
    alice_loss = alice_loss.get()
print (str(alice_loss) + str(bob_loss))

alice_model.move(crypto_provider)
bob_model.move(crypto_provider)
with torch.no_grad():
    model.fc1.weight.set_(((alice_model.fc1.weight + bob_model.fc1.weight) / 2).get())
    model.fc1.bias.set_(((alice_model.fc1.bias + bob_model.fc1.bias) / 2).get())
    model.fc2.weight.set_(((alice_model.fc2.weight + bob_model.fc2.weight) / 2).get())
    model.fc2.bias.set_(((alice_model.fc2.bias + bob_model.fc2.bias) / 2).get())

tensor(0.2145, requires_grad=True)tensor(0.2503, requires_grad=True)


In [14]:
bob._objects

{62243791484: tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4